In [1]:
# ==========================================
# 📌 1. IMPORTING LIBRARIES
# ==========================================

import pandas as pd
import numpy as np
import geoip2.database
import user_agents
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re

# To ignore warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# tqdm support for pandas
tqdm.pandas()


In [2]:
# ==========================================
# 📌 2. LOADING THE DATA (test.xlsx)
# ==========================================

# Load the provided Excel file
file_path = 'test.xlsx'  # Update path if necessary
df = pd.read_excel(file_path)

# Show the first few rows
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'test.xlsx'

In [ ]:
# ==========================================
# 📌 3. ENRICHING IP ADDRESSES (USING LOCAL GeoLite2)
# ==========================================

# Load GeoLite2 database
geoip_reader = geoip2.database.Reader('GeoLite2-City.mmdb')

def enrich_ip(ip_list):
    """
    Given a string of IPs (comma-separated), returns location data from the first IP.
    """
    if pd.isna(ip_list):
        return pd.Series([np.nan]*4, index=['ip_country', 'ip_city', 'ip_timezone', 'ip_subnet'])
    
    first_ip = ip_list.split(',')[0].strip()
    try:
        response = geoip_reader.city(first_ip)
        return pd.Series([
            response.country.name,
            response.city.name,
            response.location.time_zone,
            first_ip.rsplit('.', 1)[0] + ".0/24"  # Assume /24 subnet
        ], index=['ip_country', 'ip_city', 'ip_timezone', 'ip_subnet'])
    except Exception:
        return pd.Series([np.nan]*4, index=['ip_country', 'ip_city', 'ip_timezone', 'ip_subnet'])

# Enrich IP information
df[['ip_country', 'ip_city', 'ip_timezone', 'ip_subnet']] = df['ips'].progress_apply(enrich_ip)

# Close the reader
geoip_reader.close()


In [ ]:
# ==========================================
# 📌 4. ENRICHING USER AGENT STRINGS
# ==========================================

def parse_user_agent(ua_string):
    """
    Parses the user agent string into OS, Browser, Mobile/PC.
    """
    if pd.isna(ua_string):
        return pd.Series([np.nan]*4, index=['os_family', 'os_version', 'browser_family', 'is_mobile'])
    
    ua = user_agents.parse(ua_string)
    return pd.Series([
        ua.os.family,
        ua.os.version_string,
        ua.browser.family,
        ua.is_mobile
    ], index=['os_family', 'os_version', 'browser_family', 'is_mobile'])

# Enrich User Agent information
df[['os_family', 'os_version', 'browser_family', 'is_mobile']] = df['browser'].progress_apply(parse_user_agent)


In [ ]:
# ==========================================
# 📌 5. EXPORTING ENRICHED DATA
# ==========================================

# Save the enriched data to a CSV
df.to_csv('enriched_data.csv', index=False)
print("✅ Enriched CSV saved as 'enriched_data.csv'")


In [ ]:
# ==========================================
# 📌 6. FINDING CONNECTIONS (COMPROMISED DEVICE & USER)
# ==========================================

# Known compromised device and user
compromised_device = '91b12379-8098-457f-a2ad-a94d767797c2'
compromised_identity = '0007f265568f1abc1da791e852877df2047b3af9'

# Devices or identities matching compromised ones
device_related = df[df['device_id'] == compromised_device]
identity_related = df[df['identity'] == compromised_identity]

# Subnets associated with compromised entries
compromised_subnets = set(device_related['ip_subnet'].dropna().tolist() + identity_related['ip_subnet'].dropna().tolist())

# Entries on the same IP subnets
subnet_related = df[df['ip_subnet'].isin(compromised_subnets)]

# Combine all related entries
connected_df = pd.concat([device_related, identity_related, subnet_related]).drop_duplicates()

print(f"🔍 Total connected entries found: {len(connected_df)}")
connected_df


In [ ]:
# ==========================================
# 📌 7. VISUALIZATION (NETWORK GRAPH)
# ==========================================

# Initialize Graph
G = nx.Graph()

# Add compromised nodes
G.add_node('compromised_device', label='device', color='red')
G.add_node('compromised_identity', label='identity', color='red')

# Add connections based on the connected dataframe
for idx, row in connected_df.iterrows():
    device_node = row['device_id']
    identity_node = row['identity']
    
    G.add_node(device_node, label='device', color='blue')
    G.add_node(identity_node, label='identity', color='green')
    
    # Link device to identity
    G.add_edge(device_node, identity_node)

# Node colors
node_colors = [G.nodes[node]['color'] for node in G.nodes]

# Draw Graph
plt.figure(figsize=(14,10))
nx.draw(
    G,
    with_labels=True,
    node_color=node_colors,
    font_size=8,
    node_size=600,
    edge_color='gray',
    font_weight='bold'
)
plt.title('🔗 Connection Network: Devices and Identities')
plt.show()


In [ ]:
# ==========================================
# 📌 8. ADDITIONAL VISUALIZATION: HEATMAP
# ==========================================

# Create a pivot table (device vs identity)
# heatmap_data = pd.crosstab(connected_df['device_id'], connected_df['identity'])

# # Plot heatmap
# plt.figure(figsize=(10,6))
# sns.heatmap(heatmap_data, cmap='Blues', linewidths=0.5, annot=True, fmt='d')
# plt.title('📊 Heatmap of Device ↔ Identity Connections')
# plt.xlabel('Identity')
# plt.ylabel('Device ID')
# plt.show()
